In [1]:
import pandas as pd
from googletrans import Translator

In [10]:
#import the dataset
df = pd.read_csv("./data/interviews/interviews.csv")
df.head(1)

,Unnamed: 0,file_name,label,text
0,0,BG_Box_AleksandarPovetkin_vs_AnthonyDjoshua_NO,NO,"Аз съм в много добра форма, проведох отличен т..."


In [19]:
df.columns

Index(['Unnamed: 0', 'file_name', 'label', 'text', 'English'], dtype='object')

## Translate the dataset in English

In [12]:
#instantiate the translator and detect the language
translator = Translator()
df['English'] = df['text'].apply(translator.translate, src='bg', dest='en').apply(getattr, args=('text',))

Somehow some lines are translated, but some others are not translated into English:

In [16]:
df.loc[20,'English'][:100]

'Тренировъчният лагер почти беше свършил, оставаше ми само още една седмица спаринги.\r\nКогато се качи'

In [20]:
from ipyannotate import annotate
def display_record(record):
    display(record[1].drop(['Unnamed: 0', 'file_name', 'label', 'text']))

data = df.iterrows()


annotation = annotate(data, display=display_record)
ok, error, next, back = annotation.toolbar.buttons

In [21]:
display(annotation)

Annotation(canvas=OutputCanvas(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'English    I a…

In [54]:
!mkdir -p data/interviews/manually_translated/bg
df['Manually_Labeled'] = [task.value for task in annotation.tasks]
for index, row in df.loc[df.Manually_Labeled == 0].iterrows():
    with open('data/interviews/manually_translated/bg/interview_'+ str(index) + '.txt', 'w') as f:
        f.write(row['English'])

In [59]:
#we translate those interviews manually and put them in this directory:
!mkdir -p data/interviews/manually_translated/en

#then we change in the dataframe
for index, row in df.loc[df.Manually_Labeled == 0].iterrows():
    with open('data/interviews/manually_translated/en/interview_'+ str(index) + '.txt', 'r') as f:
        df.loc[index, 'English'] = f.read()

In [64]:
df.to_csv("./data/interviews/interviews_en.csv")